In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

import time

/home/edu/miniconda3/envs/AI-ECG/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
content = pd.read_json(path_or_buf='../data/content.jsonl',lines=True)
content = content[['Genre','Plot','ItemId']]
content = content.sort_values(['ItemId'])

targets = pd.read_csv("../data/targets.csv")
targets = targets.sort_values('UserId')
ratings = pd.read_json(path_or_buf='../data/ratings.jsonl',lines=True)

In [3]:
start = time.time()

users = targets['UserId'].unique()
movies_watched_per_user = [None]*len(users)
for i in range(len(users)):
    movies_watched_per_user[i] = (ratings[ratings['UserId'] == users[i]]['ItemId']).values 

print(time.time() - start)

270.04754424095154


In [4]:
text_data = content.to_numpy()
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)

Batches: 100%|██████████████████████████████████████████████████████████████████████| 1188/1188 [15:19<00:00,  1.29it/s]


In [5]:
cos_sim_data = pd.DataFrame(cosine_similarity(embeddings)) # item simmilarity table

In [6]:
itens_id = content['ItemId'].values

In [29]:
start = time.time()

target_users = targets['UserId'].values
target_itens = targets['ItemId'].values

ratings_final = [None]*len(target_users)

last_index = 0
start1 = time.time()

for i in range(len(users)):
    index = target_users.searchsorted(target_users[i], side='right')
    target_movies_by_user = target_itens[last_index:index]
    #print(target_movies_by_user)
    j = last_index
    for movie in target_movies_by_user:
        index_movie = itens_id.searchsorted(movie, side='right') - 1 
        score = 0
        
        if len(movies_watched_per_user[i]) > 10:
            
            movie_list = movies_watched_per_user[i][:10]
        else:
            movie_list = movies_watched_per_user[i]
        
        for movie_watched in movie_list:
            index_movie_watched = itens_id.searchsorted(movie_watched, side='right') - 1 
            score += cos_sim_data[index_movie][index_movie_watched]
        ratings_final[j] = score
        j+=1
print(time.time() - start)

639.4558815956116


In [19]:
np.save('ratings.npy', np.array(ratings_final))

In [25]:
predictions = targets
predictions['Ratings'] = ratings_final
predictions.head(20)

,UserId,ItemId,Ratings
0,0006246bee,01d2404d4c,96.015408
72,0006246bee,cc641b84fe,98.600180
71,0006246bee,c64177b66b,102.054995
70,0006246bee,c5388f07fc,100.187000
69,0006246bee,c1ee6829f5,101.682531
68,0006246bee,bf95622d07,82.638502
67,0006246bee,bf6a723c91,101.001041
66,0006246bee,bb3396574c,99.034706
65,0006246bee,b804592040,81.654915
64,0006246bee,b78488f85a,85.462496


In [24]:
predictions = pd.DataFrame(data=predictions,columns=['UserId', 'ItemId','Rating'])
predictions = predictions.sort_values(['UserId', 'Rating'], ascending=[True, False])
ratings = predictions["Rating"]
predictions = predictions.drop("Rating",axis=1)

predictions.to_csv("sub.csv",index=False)
    
print("submission done")

submission done
